In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
project_folder_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database'
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *
from utils.pdf_functs import *

/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database


/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Database and directories

In [2]:
source_db_file_name = 'cev-database-coordinates-v1.db'
source_db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', source_db_file_name)
destination_db_file_name = 'cev-database-reports-v1.db'
destination_db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', destination_db_file_name)

## 1) Define a dataframe with download coordinates
The dataframe must contain 1 row per `comuna` meaning 348 rows in total.

In [3]:
query = f"""
        SELECT * FROM evals_summary;
        """
print(query)


        SELECT * FROM evals_summary;
        


In [4]:
df = create_dataframe_from_query(source_db_file_path, query)
df.head()

,eval_id,comuna_id,region_id,tipo_evaluacion,identificacion_vivienda,tipologia,comuna,proyecto,CE,CEE
0,fec88ef2-ceb6-5143-850c-473800d9bcfa,5,1,1,29475-Piso 2 - sur-oriente TORRE A,Departamento,Alto Hospicio,CONSTRUCCIÓN DE OBRA NUEVA 64 DEPTOS Y 1 SEDE ...,F,F
1,9f827408-4a78-5bc5-b9d7-76d51ea16905,5,1,1,41623-BLOCK 2_DEPTO 203,Departamento,Alto Hospicio,CONJUNTO HABITACIONAL VISTA MANSA 1,E,E
2,1c89307d-c14f-50f4-ae63-8bd19cb8d0ee,5,1,1,41669-BLOCK 4_DEPTO 304,Departamento,Alto Hospicio,CONJUNTO HABITACIONAL VISTA MANSA 1,E,E
3,3e84642c-1907-5729-8c34-11e8ac165662,5,1,1,41646-BLOCK 3_DEPTO 503,Departamento,Alto Hospicio,CONJUNTO HABITACIONAL VISTA MANSA 1,E,E
4,fb43e1da-18ed-5106-bb94-7c937e5015bf,5,1,1,"80106-Depto 303, SurOeste, piso 3, Block 2",Departamento,Alto Hospicio,Condominio Pampa Perdiz I,F,F


Set additional features to track the status of downloads as well as times and file names

insert_unique_rows_from_dataframe(destination_db_file_path, 'evals_summary', df, unique_columns=['eval_id'])

## 3) Check first if any pdf files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [5]:
# Define the directory path
pdf_files_dir = os.path.join(project_folder_path, 'data', 'raw', '3_evals_reports', 'pdf_files')

# Check if the directory exists
if not os.path.exists(pdf_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(pdf_files_dir)
    print(f"Directory '{pdf_files_dir}' created.")

# List all files in the directory
pdf_file_paths = find_pdf_files(pdf_files_dir)
print(f'The directory: {pdf_files_dir} contains {len(pdf_file_paths)} out of {df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database/data/raw/3_evals_reports/pdf_files contains 12383 out of 72412


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

### Scrape pdf files: 1 file per evaluacion

In [6]:
### PDF Summary

In [7]:
summary_df = pd.DataFrame(data=pdf_file_paths, columns=['pdf_file_path'])
summary_df[['directory', 'pdf_file_name']] = summary_df['pdf_file_path'].str.rsplit('/', n=1, expand=True)
#summary_df['pdf_file_name'] = summary_df['pdf_file_name'].str.replace('.pdf', '')
summary_df[['region_id', 'comuna_id', 'tipo_evaluacion', 'eval_id']] = summary_df['pdf_file_name'].str.replace('.pdf', '').str.rsplit('_', n=0, expand=True)
#summary_df.drop(columns=['pdf_file_path', 'directory'], inplace=True)
summary_df.T

,0,1,2,3,4,5,6,7,8,9,...,12373,12374,12375,12376,12377,12378,12379,12380,12381,12382
pdf_file_path,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...
directory,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...
pdf_file_name,1_113_1_01e92e4e-f6b7-514e-a256-06264f380675.pdf,1_113_1_0884c79b-e7a6-5c70-89b6-a54faafc65ec.pdf,1_113_1_0b05bbd6-37b3-5af0-a2a9-2efca779001f.pdf,1_113_1_14fbb359-7c44-5e81-9d3f-0d6fa2364b76.pdf,1_113_1_16035bae-c656-5e37-b3f9-11e3c97a980c.pdf,1_113_1_2067ae87-e8b8-519c-b707-e72f906aed36.pdf,1_113_1_28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04.pdf,1_113_1_2d77b393-c5ac-5329-9d29-4371de25d2cd.pdf,1_113_1_2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8.pdf,1_113_1_34860214-2325-5c64-8f0a-bfbf553d832c.pdf,...,9_92_2_ba5d8ef0-7495-5bd2-813c-94dde5a83486.pdf,9_92_2_bbbd5818-8420-5c1a-95fd-e2947b380659.pdf,9_92_2_c7e087b6-2adf-5cf3-872f-38f779af0ca3.pdf,9_92_2_d2f4f0aa-df00-57fa-ad46-fbc06cad83df.pdf,9_92_2_da89e248-c999-5bbe-8826-dfec3aea7bd6.pdf,9_92_2_dabac87f-fe04-5d25-a012-6484392113dd.pdf,9_92_2_dc5a945d-cd03-5435-bcf5-c87971e75f5d.pdf,9_92_2_ed98cb99-81bb-54cf-87c8-4d23772db070.pdf,9_92_2_f4613a79-fa71-5941-9d89-7f8331ad7fa7.pdf,9_92_2_f9f4b8b7-2830-5b59-860d-d51bbc3c8964.pdf
region_id,1,1,1,1,1,1,1,1,1,1,...,9,9,9,9,9,9,9,9,9,9
comuna_id,113,113,113,113,113,113,113,113,113,113,...,92,92,92,92,92,92,92,92,92,92
tipo_evaluacion,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
eval_id,01e92e4e-f6b7-514e-a256-06264f380675,0884c79b-e7a6-5c70-89b6-a54faafc65ec,0b05bbd6-37b3-5af0-a2a9-2efca779001f,14fbb359-7c44-5e81-9d3f-0d6fa2364b76,16035bae-c656-5e37-b3f9-11e3c97a980c,2067ae87-e8b8-519c-b707-e72f906aed36,28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04,2d77b393-c5ac-5329-9d29-4371de25d2cd,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,34860214-2325-5c64-8f0a-bfbf553d832c,...,ba5d8ef0-7495-5bd2-813c-94dde5a83486,bbbd5818-8420-5c1a-95fd-e2947b380659,c7e087b6-2adf-5cf3-872f-38f779af0ca3,d2f4f0aa-df00-57fa-ad46-fbc06cad83df,da89e248-c999-5bbe-8826-dfec3aea7bd6,dabac87f-fe04-5d25-a012-6484392113dd,dc5a945d-cd03-5435-bcf5-c87971e75f5d,ed98cb99-81bb-54cf-87c8-4d23772db070,f4613a79-fa71-5941-

In [8]:
summary_df['is_pdf_file_valid'] = None
summary_df['version_evaluacion'] = None
summary_df['codigo_evaluacion'] = None

In [9]:
summary_df = summary_df.head(10)

In [10]:
for index, row in summary_df.iterrows():
    print(f'Report {index+1} out of {summary_df.shape[0]}')
    pdf_file_path = row['pdf_file_path']
    try:
        pdf_report = fitz.open(pdf_file_path)
        
        if pdf_report.page_count == 4:
            summary_df.loc[index, 'is_pdf_file_valid'] = True
            summary_df.loc[index, 'version_evaluacion'] = '1'
            page_number = 1  # Page number (starting from 0)
            page = pdf_report[page_number]
            # Código evaluación energética
            area_coordinates = (60.2, 21.5, 80.3, 25.3)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
            extracted_text = extract_text_from_area(page, area_coordinates)
            codigo_evaluacion = extracted_text.split('\n')[-1]
            summary_df.loc[index, 'codigo_evaluacion'] = codigo_evaluacion
            
        elif pdf_report.page_count == 7:
            summary_df.loc[index, 'is_pdf_file_valid'] = True
            summary_df.loc[index, 'version_evaluacion'] = '2'
            page_number = 2  # Page number (starting from 0)
            page = pdf_report[page_number]
            # Código evaluación energética
            area_coordinates = (62.3, 30.7, 88.1, 35.1)  # Coordinates of the area to extract text from: (x1, y1, x2, y2)
            extracted_text = extract_text_from_area(page, area_coordinates)
            codigo_evaluacion = extracted_text
            summary_df.loc[index, 'codigo_evaluacion'] = codigo_evaluacion
            
        else:        
            summary_df.loc[index, 'is_pdf_file_valid'] = False
            summary_df.loc[index, 'version_evaluacion'] = None 
            summary_df.loc[index, 'codigo_evaluacion'] = None           

        
    except Exception as e:
        print(f"FileDataError: cannot open broken document: {e}")
        summary_df.loc[index, 'is_pdf_file_valid'] = False
        summary_df.loc[index, 'version_evaluacion'] = None         
            
summary_df.T

Report 1 out of 10
Report 2 out of 10
Report 3 out of 10
Report 4 out of 10
Report 5 out of 10
Report 6 out of 10
Report 7 out of 10
Report 8 out of 10
Report 9 out of 10
Report 10 out of 10


,0,1,2,3,4,5,6,7,8,9
pdf_file_path,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...
directory,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...,/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-...
pdf_file_name,1_113_1_01e92e4e-f6b7-514e-a256-06264f380675.pdf,1_113_1_0884c79b-e7a6-5c70-89b6-a54faafc65ec.pdf,1_113_1_0b05bbd6-37b3-5af0-a2a9-2efca779001f.pdf,1_113_1_14fbb359-7c44-5e81-9d3f-0d6fa2364b76.pdf,1_113_1_16035bae-c656-5e37-b3f9-11e3c97a980c.pdf,1_113_1_2067ae87-e8b8-519c-b707-e72f906aed36.pdf,1_113_1_28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04.pdf,1_113_1_2d77b393-c5ac-5329-9d29-4371de25d2cd.pdf,1_113_1_2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8.pdf,1_113_1_34860214-2325-5c64-8f0a-bfbf553d832c.pdf
region_id,1,1,1,1,1,1,1,1,1,1
comuna_id,113,113,113,113,113,113,113,113,113,113
tipo_evaluacion,1,1,1,1,1,1,1,1,1,1
eval_id,01e92e4e-f6b7-514e-a256-06264f380675,0884c79b-e7a6-5c70-89b6-a54faafc65ec,0b05bbd6-37b3-5af0-a2a9-2efca779001f,14fbb359-7c44-5e81-9d3f-0d6fa2364b76,16035bae-c656-5e37-b3f9-11e3c97a980c,2067ae87-e8b8-519c-b707-e72f906aed36,28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04,2d77b393-c5ac-5329-9d29-4371de25d2cd,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,34860214-2325-5c64-8f0a-bfbf553d832c
is_pdf_file_valid,True,True,True,True,True,True,True,True,True,True
version_evaluacion,2,1,1,1,1,1,1,1,2,1
codigo_evaluacion,876c2f2018,4125252016,fc03342016,32929f2016,c1ae042017,5616122017,ba70de2016,2a34b32017,83c33a2018,281f912016


In [11]:
# Drop specified columns
columns_to_drop = ['pdf_file_path', 'directory']
out_df = summary_df.drop(columns=columns_to_drop).dropna().reset_index(drop=True)
out_df = out_df[['eval_id', 'comuna_id', 'region_id', 'tipo_evaluacion', 'version_evaluacion', 'codigo_evaluacion', 'pdf_file_name', 'is_pdf_file_valid']]          
out_df

,eval_id,comuna_id,region_id,tipo_evaluacion,version_evaluacion,codigo_evaluacion,pdf_file_name,is_pdf_file_valid
0,01e92e4e-f6b7-514e-a256-06264f380675,113,1,1,2,876c2f2018,1_113_1_01e92e4e-f6b7-514e-a256-06264f380675.pdf,True
1,0884c79b-e7a6-5c70-89b6-a54faafc65ec,113,1,1,1,4125252016,1_113_1_0884c79b-e7a6-5c70-89b6-a54faafc65ec.pdf,True
2,0b05bbd6-37b3-5af0-a2a9-2efca779001f,113,1,1,1,fc03342016,1_113_1_0b05bbd6-37b3-5af0-a2a9-2efca779001f.pdf,True
3,14fbb359-7c44-5e81-9d3f-0d6fa2364b76,113,1,1,1,32929f2016,1_113_1_14fbb359-7c44-5e81-9d3f-0d6fa2364b76.pdf,True
4,16035bae-c656-5e37-b3f9-11e3c97a980c,113,1,1,1,c1ae042017,1_113_1_16035bae-c656-5e37-b3f9-11e3c97a980c.pdf,True
5,2067ae87-e8b8-519c-b707-e72f906aed36,113,1,1,1,5616122017,1_113_1_2067ae87-e8b8-519c-b707-e72f906aed36.pdf,True
6,28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04,113,1,1,1,ba70de2016,1_113_1_28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04.pdf,True
7,2d77b393-c5ac-5329-9d29-4371de25d2cd,113,1,1,1,2a34b32017,1_113_1_2d77b393-c5ac-5329-9d29-4371de25d2cd.pdf,True
8,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,113,1,1,2,83c33a2018,1_113_1_2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8.pdf,True
9,34860214-2325-5c64-8f0a-bfbf553d832c,113,1,1,1,281f912016,1_113_1_34860214-2325-5c64-8f0a-bfbf553d832c.pdf,True


In [12]:
out_df['comuna_id'] = out_df['comuna_id'].astype(int)
out_df['region_id'] = out_df['region_id'].astype(int)
out_df['tipo_evaluacion'] = out_df['tipo_evaluacion'].astype(int)
out_df['version_evaluacion'] = out_df['version_evaluacion'].astype(int)
out_df['is_pdf_file_valid'] = out_df['is_pdf_file_valid'].astype(int)
out_df

,eval_id,comuna_id,region_id,tipo_evaluacion,version_evaluacion,codigo_evaluacion,pdf_file_name,is_pdf_file_valid
0,01e92e4e-f6b7-514e-a256-06264f380675,113,1,1,2,876c2f2018,1_113_1_01e92e4e-f6b7-514e-a256-06264f380675.pdf,1
1,0884c79b-e7a6-5c70-89b6-a54faafc65ec,113,1,1,1,4125252016,1_113_1_0884c79b-e7a6-5c70-89b6-a54faafc65ec.pdf,1
2,0b05bbd6-37b3-5af0-a2a9-2efca779001f,113,1,1,1,fc03342016,1_113_1_0b05bbd6-37b3-5af0-a2a9-2efca779001f.pdf,1
3,14fbb359-7c44-5e81-9d3f-0d6fa2364b76,113,1,1,1,32929f2016,1_113_1_14fbb359-7c44-5e81-9d3f-0d6fa2364b76.pdf,1
4,16035bae-c656-5e37-b3f9-11e3c97a980c,113,1,1,1,c1ae042017,1_113_1_16035bae-c656-5e37-b3f9-11e3c97a980c.pdf,1
5,2067ae87-e8b8-519c-b707-e72f906aed36,113,1,1,1,5616122017,1_113_1_2067ae87-e8b8-519c-b707-e72f906aed36.pdf,1
6,28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04,113,1,1,1,ba70de2016,1_113_1_28ee85b8-3f92-5fdb-a92c-5d9fc49ccc04.pdf,1
7,2d77b393-c5ac-5329-9d29-4371de25d2cd,113,1,1,1,2a34b32017,1_113_1_2d77b393-c5ac-5329-9d29-4371de25d2cd.pdf,1
8,2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8,113,1,1,2,83c33a2018,1_113_1_2eb2a9b8-18d6-587f-bfbf-d13162b0a6d8.pdf,1
9,34860214-2325-5c64-8f0a-bfbf553d832c,113,1,1,1,281f912016,1_113_1_34860214-2325-5c64-8f0a-bfbf553d832c.pdf,1


In [13]:
insert_unique_rows_from_dataframe(destination_db_file_path, 'reports_summary', out_df, unique_columns=['eval_id'])

Rows inserted into table 'reports_summary' successfully.
